In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import TimeDistributed
from keras.layers import ConvLSTM2D
from keras.utils import to_categorical
from matplotlib import pyplot
from sklearn.model_selection import train_test_split

2022-05-19 15:55:01.418195: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-19 15:55:01.418232: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
def summarize_results(scores):
	print(scores)
	m, s = np.mean(scores), np.std(scores)
	print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))

In [17]:
# load data
x = pd.read_csv('waveDataDrop.csv')
y = pd.read_csv('y.csv', header=None)
trainX, testX, trainy, testy=train_test_split(x,y)
trainy = trainy.to_numpy()
trainX = trainX.to_numpy()
testX = testX.to_numpy()
testy = testy.to_numpy()
scores = list()

In [18]:
verbose, epochs, batch_size = 0, 25, 64
n_timesteps, n_features, n_outputs = trainX.shape[1], 1, trainy.shape[0]
# reshape into subsequences (samples, time steps, rows, cols, channels)
n_steps, n_length = 4, 32
trainX = trainX.reshape((trainX.shape[0], 1, 1, trainX.shape[1], 1))
testX = testX.reshape((testX.shape[0], 1, 1, testX.shape[1], 1))
# trainX = trainX.reshape((trainX.shape[0], n_steps, 1, n_length, n_features))
# testX = testX.reshape((testX.shape[0], n_steps, 1, n_length, n_features))
# trainX = trainX.reshape((1, 1, trainX.shape[0], trainX.shape[1], 1))
# testX = testX.reshape((1, 1, testX.shape[0], testX.shape[1], 1))

In [19]:
# reshape into subsequences (samples, time steps, rows, cols, channels) https://machinelearningmastery.com/how-to-develop-rnn-models-for-human-activity-recognition-time-series-classification/
# Samples: n, for the number of windows in the dataset.
# Time: 4, for the four subsequences that we split a window of 128 time steps into.
# Rows: 1, for the one-dimensional shape of each subsequence.
# Columns: 32, for the 32 time steps in an input subsequence.
# Channels: 9, for the nine input variables.
trainX.shape

(15, 1, 1, 1679, 1)

In [22]:
# define model
model = Sequential()
model.add(ConvLSTM2D(filters=32, kernel_size=(9,1), activation='relu', input_shape=(1, trainX.shape[0], 1, 1))) # TODO Error
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(n_outputs, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit network
model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=0)
# evaluate model
_, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)


score = score * 100.0
print('>#%d: %.3f' % (0+1, score))
scores.append(score)

summarize_results(scores)

ValueError: in user code:

    File "/home/anrath/miniconda3/envs/bio/lib/python3.10/site-packages/keras/engine/training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "/home/anrath/miniconda3/envs/bio/lib/python3.10/site-packages/keras/engine/training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/anrath/miniconda3/envs/bio/lib/python3.10/site-packages/keras/engine/training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "/home/anrath/miniconda3/envs/bio/lib/python3.10/site-packages/keras/engine/training.py", line 889, in train_step
        y_pred = self(x, training=True)
    File "/home/anrath/miniconda3/envs/bio/lib/python3.10/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/home/anrath/miniconda3/envs/bio/lib/python3.10/site-packages/keras/e